<a href="https://colab.research.google.com/github/prithsha/erav2/blob/main/era_v2_s2_alok_singh_chauhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Importing required libraries 
# torch is a deep learning framework well integrated with python development
import torch

#  pre-built classes and functions specifically designed for creating neural networks.
# nn.Conv2d form the basic building blocks of neural networks
# nn.Module provide a framework for organizing layers and their parameters
import torch.nn as nn

# torch.nn.functional provides a collection functions for neural networks like:
# 1- activation functions like ReLU 
# 2- Loss functions like L1Loss
# 3- Convolution operation like Conv2d
import torch.nn.functional as F

# optim provides algorithms and tools for optimizing the parameters of your deep learning models.
# Example Optimizers: Stochastic Gradient Descent (SGD), Adam, 
import torch.optim as optim
# torchvision library designed for computer vision tasks and also provides data sets
from torchvision import datasets, transforms
%pip install torchsummary
# !pip install torchsummary
# tourchsummary library helps in visualizing and analyzing the architecture of deep learning models.
from torchsummary import summary

print(f"torch version {torch.__version__}")

torch version 2.2.0+cu118



In [62]:
# Checking if support for cuda is available or not so that we can use GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [63]:
batch_size = 128
# 128 image samples in one batch
# Batching: Divides data into smaller batches, reducing memory usage and enabling larger datasets and improved speed.

# transforms.Compose process image data prior to feeding it into your models.
# operations like Merges multiple transformation operations into a single entity, normalizing or standardizing the pixel values
# transforms.Normalize we are passing the mean and standard deviation for a gray scale
# Values are 0.1307 indicate that preprocessing is already applied on gray scale otherwise pixel value ranges from 0(Black)-255(White)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Dataloader manages efficient data loading and managing datasets 
# Now we have got two instances of DataLoader. One for training and other for testing



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [64]:
# Data r_in, n_in, etc below calculated manually for all layers. 
# later created a excel to  calculate that. Available at github: https://github.com/prithsha/erav2
# File name "Receptive field out-pixel calculator.xlsx"

class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in:3 , n_in:28 , j_in:1 , s:1 , r_out: 5, n_out:28 , j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:28 , j_in:1 , s:2 , r_out: 6, n_out:14 , j_out:2
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in:6, n_in:14 , j_in:2 , s:1 , r_out: 10, n_out:14 , j_out:2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in:10 , n_in:14 , j_in:2 , s:1 , r_out:14 , n_out:14 , j_out:2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in: 14, n_in:14 , j_in:2 , s:2 , r_out:16 , n_out:7 , j_out:4
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in:16 , n_in:7 , j_in:4 , s:1 , r_out:24 , n_out:5 , j_out:4
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in:24 , n_in:5 , j_in: 4, s:1 , r_out:32 , n_out:3 , j_out:4
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in:32 , n_in:3 , j_in:4 , s:1 , r_out:40 , n_out:1 , j_out:4
    self.conv7 = nn.Conv2d(1024, 10, 3)
# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
# Function how data is passed from one layer to another
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    # We dont need to calculate relu again as after conv7 we have 10 channels.
    # Applying relu on this will reduce negative numbers to 0 which will remove capability of some feature identification 
    # x = F.relu(x) # this is the last step. Think what ReLU does to our results at this stage!
    # print(f"Before reshape {x.shape}")
    x = x.view(-1, 10)
    # print(f"After reshape {x.shape}")

    # below function performs two operations

    # softmax calculates the exponentials of all elements, sums them up and divide each element by SUM. So its normalized probability
    # log compress large numbers and spreads smaller ones
    return F.log_softmax(x)


In [65]:
# creating instance of neural network and sending to hardware
model = FirstDNN().to(device)

In [66]:
# Summarizes each layer (batch dimension, number of output channels, height , width of result feature map )
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

C:\Users\1430647\AppData\Local\Temp\1\ipykernel_50868\2523713694.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [67]:
from tqdm import tqdm
# start training on training data loader set
def train(model, device, train_loader, optimizer, epoch):
    # We are setting training mode to true
    model.train()
    # Progress bar
    pbar = tqdm(train_loader)
    # # Iterate over batches during training
    for batch_idx, (data, target) in enumerate(pbar):
        # data: A tensor representing the image data for the current batch.
        # target: A tensor containing the corresponding labels for the images in the batch.
        # Sending to GPU
        data, target = data.to(device), target.to(device)
        #  Resets the gradients of model parameters to zero before starting backpropagation.
        optimizer.zero_grad()
        # Forward pass through the model to compute predictions for the current batch.
        output = model(data)
        loss = F.nll_loss(output, target)
        #  Performs backpropagation to compute gradients of the loss
        loss.backward()
        # Updates model parameters based on the calculated gradients.
        optimizer.step()
        # Updates the progress bar with current loss and batch ID.
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


# start test on test data loader set
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [68]:
#optimizer function
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Starting
for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]C:\Users\1430647\AppData\Local\Temp\1\ipykernel_50868\2523713694.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.055574432015419006 batch_id=468: 100%|██████████| 469/469 [00:32<00:00, 14.56it/s]



Test set: Average loss: 0.0552, Accuracy: 9819/10000 (98%)

